In [3]:
import cv2 as cv
import numpy as np
import os

def filtrar_huellas(imagen):
    # Convertir la imagen a escala de grises
    imagen_gris = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
    # Aplicar un filtro de mediana para eliminar ruido
    imagen_filtrada = cv.GaussianBlur(imagen_gris, (5, 5), 0)
    # Aplicar un filtro de Canny para detectar bordes
    imagen_bordes = cv.Canny(imagen_filtrada, 100, 200)
    # Aplicar un filtro de dilatación para cerrar los bordes
    kernel = np.ones((5,5), np.uint8)
    imagen_dilatada = cv.dilate(imagen_bordes, kernel, iterations=1)
    return imagen_dilatada

sample_path = r"\BaseDatosNISTsd09_pr"
sample_path = os.getcwd()+sample_path
files= os.listdir(sample_path)

for file in files:
    dir = sample_path +'\\'
    dir = dir + file + '\\'
    file2= os.listdir(dir)
    ruta=dir+'\\'+file2[0]
    sample = cv.imread(ruta) #muestras para tomar de referencia
    sample = filtrar_huellas(sample)

# mostrar la imagen
cv.imshow('Huellas', sample)
cv.waitKey(0)
cv.destroyAllWindows()
